<a href="https://colab.research.google.com/github/chetools/STEMUnleashed2025/blob/main/InvestmentSimulations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from plotly.subplots import make_subplots
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go

In [2]:
p=np.array([9/13, 4/13])
r=np.array([1.1, 0.78])

In [3]:
def log_ret(f, p, r):
    return np.sum(p*np.log(1+f*(r-1)))

In [4]:
fs = np.linspace(0,0.99,100)
rets = np.array([np.exp(log_ret(f,p,r)) for f in fs])
fig=make_subplots()
fig.add_scatter(x=fs, y=rets, mode='lines')
fig.update_layout(width=800, height=600, template='plotly_dark')

In [5]:
ticker_symbol = 'SPY'  # P&G ticker symbol
end_date = pd.to_datetime('today').strftime('%Y-%m-%d')
start_date = (pd.to_datetime('today') - pd.DateOffset(years=50)).strftime('%Y-%m-%d')
stock_data = yf.download(ticker_symbol, start=start_date, end=end_date)
orig_closing = stock_data.loc[:,'Close'].values.squeeze()
percent_changes=orig_closing[1:]/orig_closing[:-1]



/tmp/ipython-input-5-1310897652.py:4: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


In [6]:
p,edges = np.histogram(percent_changes,bins=500)
p=p/np.sum(p)
mid = (edges[1:]+edges[:-1])/2

In [7]:
fs = np.linspace(0,0.99,100)
rets = np.array([np.exp(log_ret(f,p,mid)) for f in fs])
fig=make_subplots()
fig.add_scatter(x=fs, y=rets, mode='lines')
fig.update_layout(width=800, height=600, template='plotly_dark')

In [8]:
ndays = [100, 200, 500, 1000, 2000, 5000]


In [9]:
def return_distribution(ndays, nsamples=1000):
   returns = np.prod(percent_changes[np.argsort(np.random.uniform(size=(nsamples, percent_changes.size)),axis=1)[:, :ndays]],axis=1)
   return returns

In [10]:
nday_returns=[return_distribution(nday, nsamples=1000) for nday in ndays]

In [11]:
nsamples=1000
[np.sum(return_distribution(nday,nsamples=nsamples)<1).item()/nsamples for nday in ndays]

[0.361, 0.313, 0.199, 0.105, 0.04, 0.0]

In [12]:
fig=make_subplots(rows=3,cols=2, subplot_titles=[f'Days: {nday}' for nday in ndays])
for i, nday in enumerate(ndays):
    fig.add_histogram(x=nday_returns[i], row=(i % 3)+1, col=i//3 + 1)
fig.update_layout(width=600, height=900, showlegend=False)
fig.show()

In [ ]:
np.rolling

In [27]:
returns_3day_seq=np.prod(np.lib.stride_tricks.sliding_window_view(percent_changes,3),axis=1)
returns_3day_random=np.prod(percent_changes[np.random.randint(0,percent_changes.size,size=(percent_changes.size,3))],axis=1)
returns_10day_seq=np.prod(np.lib.stride_tricks.sliding_window_view(percent_changes,10),axis=1)
returns_10day_random=np.prod(percent_changes[np.random.randint(0,percent_changes.size,size=(percent_changes.size,10))],axis=1)

In [82]:
returns_20day_seq=np.prod(np.lib.stride_tricks.sliding_window_view(percent_changes,20),axis=1)
returns_20day_random=np.prod(percent_changes[np.random.randint(0,percent_changes.size,size=(percent_changes.size,20))],axis=1)

In [70]:
prob_3day_seq=np.histogram(returns_3day_seq,bins=500,density=True)
prob_3day_random=np.histogram(returns_3day_random,bins=500,density=True)

In [71]:
prob_10day_seq=np.histogram(returns_10day_seq,bins=500,density=True)
prob_10day_random=np.histogram(returns_10day_random,bins=500,density=True)

In [83]:
prob_20day_seq=np.histogram(returns_20day_seq,bins=500,density=True)
prob_20day_random=np.histogram(returns_20day_random,bins=500,density=True)

In [84]:
fig=make_subplots()
fig.add_scatter(x=(prob_3day_seq[1][0:-1]+prob_3day_seq[1][1:])/2, y=prob_3day_seq[0], name='3day_sequence')
fig.add_scatter(x=(prob_3day_random[1][0:-1]+prob_3day_random[1][1:])/2, y=prob_3day_random[0], name='3day_random')
fig.add_scatter(x=(prob_10day_seq[1][0:-1]+prob_10day_seq[1][1:])/2, y=prob_10day_seq[0], name='10day_sequence')
fig.add_scatter(x=(prob_10day_random[1][0:-1]+prob_10day_random[1][1:])/2, y=prob_10day_random[0], name='10day_random')
fig.add_scatter(x=(prob_20day_seq[1][0:-1]+prob_20day_seq[1][1:])/2, y=prob_20day_seq[0], name='20day_sequence')
fig.add_scatter(x=(prob_20day_random[1][0:-1]+prob_20day_random[1][1:])/2, y=prob_20day_random[0], name='20day_random')
# fig.update_xaxes(range=(0.8,1.2))
# fig.update_yaxes(range=(0,800))
fig.update_layout(width=1200, height=800)